# 2. OpenAI の チャット API の基礎


## 2.3. 入出力の長さの制限や料金に影響する「トークン」


### トークン


In [ ]:
%pip install tiktoken==0.7.0

In [ ]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

### Tokenizer と tiktoken の紹介


In [ ]:
import tiktoken

text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

## 2.4. Chat Completions API を試す環境の準備


### OpenAI の API キーの準備


In [ ]:
import os

## 2.5. Chat Completions API のハンズオン


### OpenAI のライブラリ


#### 【注意】既知のエラーについて

openai パッケージが依存する httpx のアップデートにより、`openai==1.40.6` を使用する箇所で `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'` というエラーが発生するようになりました。

このエラーは、`!pip install httpx==0.27.2` のように、httpx の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install httpx==0.27.2` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [ ]:
%pip install httpx==0.27.2

### Chat Completions API の呼び出し


In [ ]:
import boto3

client = boto3.client("bedrock-runtime")

response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "You are a helpful assistant."}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "こんにちは！私はジョンと言います！"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

### 会話履歴を踏まえた応答を得る


In [ ]:
response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "You are a helpful assistant."}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "こんにちは！私はジョンと言います！"}],
        },
        {
            "role": "assistant",
            "content": [{"text": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？"}],
        },
        {
            "role": "user",
            "content": [{"text": "私の名前が分かりますか？"}],
        },
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

### ストリーミングで応答を得る


In [ ]:
response = client.converse_stream(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "You are a helpful assistant."}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "こんにちは！私はジョンと言います！"}],
        }
    ]
)

for chunk in response["stream"]:
    if "contentBlockDelta" in chunk:
        text = chunk["contentBlockDelta"]["delta"]["text"]
        print(text, end="")

### JSON モード


In [ ]:
import boto3

client = boto3.client("bedrock-runtime")

response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}'}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "昔々あるところにおじいさんとおばあさんがいました"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

### Vision（画像入力）


In [ ]:
# from openai import OpenAI

# client = OpenAI()

import boto3
import urllib.request
import base64

client = boto3.client("bedrock-runtime")

image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

with urllib.request.urlopen(image_url) as response:
    image_data = response.read()

response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "Image1: "},
                {"image": {"format": "png", "source": {"bytes": image_data}}},
                {"text": "画像を説明してください。"},
            ],
        }
    ],
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

### （コラム）Completions API


In [ ]:
import boto3

client = boto3.client("bedrock-runtime")

response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    system=[{"text": "You are a helpful assistant."}],
    messages=[
        {
            "role": "user",
            "content": [{"text": "こんにちは！私はジョンと言います！"}],
        }
    ]
)

response_text = response["output"]["message"]["content"][0]["text"]
print(response_text)

## 2.6. Function calling


### Function calling のサンプルコード


In [ ]:
import json


def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [ ]:
tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "unit": {
                                "type": "string", 
                                "enum": ["celsius", "fahrenheit"]
                            },
                        },
                        "required": [
                            "location"
                        ]
                    }
                }
            }
        }
    ]
}

In [ ]:
import boto3

client = boto3.client("bedrock-runtime")

messages = [
    {
        "role": "user",
        "content": [{"text": "How's the weather in Tokyo today?"}],
    }
]

response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    messages=messages,
    toolConfig=tool_config
)

print(json.dumps(response, indent=2))

In [ ]:
response_message = response['output']['message']
messages.append(response_message)

In [ ]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 使いたい関数は複数あるかもしれないのでループ
for tool_request in response['output']['message']['content']:
    # 関数を実行
    function_name = tool_request['toolUse']['name']
    function_to_call = available_functions[function_name]
    function_args = tool_request['toolUse']['input']
    function_response = json.loads(
        function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
    )
    
    print(function_response)

    # 関数の実行結果を会話履歴としてmessagesに追加
    messages.append(
        {
            "role": "user",
            "content": [
                {
                    "toolResult": 
                        {
                            "toolUseId": tool_request['toolUse']['toolUseId'],
                            "content": [{"json": function_response}]
                        }

                }
            ]
        }
    )

In [ ]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

In [ ]:
second_response = client.converse(
    modelId="apac.anthropic.claude-3-haiku-20240307-v1:0",
    messages=messages,
    toolConfig=tool_config
)
output_message = response['output']['message']

print(json.dumps(second_response, indent=2))